In [ ]:
%run _00config.ipynb
%run main_miky1.ipynb
%run main_miky2.ipynb

In [ ]:
sns.set_style("darkgrid")

In [ ]:
import random
random.seed(42)

## 4. Batch-Run

In [ ]:
N_agents = 40
Price = 10
Last_Price = Price
p_savers = 0.20
p_followers = 0.10
p_whales = 0.05
savers_sentiment = 0.60
p_passing_followers = 0.7
p_passing_whales = 0.5

prm_grid = {
    'N_agents': N_agents,
    'Price': Price,
    'p_savers': p_savers,
    'p_followers': p_followers,
    'p_whales': p_whales,
    'p_passing_followers': p_passing_followers,
    'p_passing_whales': p_passing_whales,
    'savers_sentiment': np.arange(0.20, 1.00, 0.10)
}

In [ ]:
results = mesa.batch_run(
    Market_Model_4,
    parameters=prm_grid,
    iterations=100,
    max_steps=200,
    number_processes=1,
    data_collection_period=1,
    display_progress=False
)

In [ ]:
results_df = pd.DataFrame(results)
display(results_df)

In [ ]:
sns.relplot(
    data = results_df, kind = 'line',
    x = 'Step', y = 'Price', hue = 'savers_sentiment',
    units = 'iteration', estimator = None
)

In [ ]:
# this is a plot with mean values for each model parameter, with 95% confidence interval error band
sns.relplot(
    data = results_df, kind = 'line',
    x = 'Step', y = 'Price', hue = 'savers_sentiment'
)

In [ ]:
_ = results_df[ results_df['savers_sentiment'] == results_df['savers_sentiment'].unique()[1]]

In [ ]:
display(_)

In [ ]:
# But we are interested in a mean + std dev plot

In [ ]:
# Take a look at all runs for a given DOF, in this case for a given value of savers_sentiment
sns.relplot(
    data = _, kind = "line",
    x = "Step", y = "Price", hue = 'iteration', palette="ch:r=-.5,l=.75",
    estimator = None
)

In [ ]:
sns.relplot(
    data = _[ _['iteration'] == 5], kind="line",
    x="Step", y="Price",
)

In [ ]:
_ = results_df[ results_df['savers_sentiment'] == results_df['savers_sentiment'].unique()[2]]

it = _['iteration'].unique().tolist()
it.pop(0)

_df = _[ _['iteration'] == 0].drop(columns = ['iteration'])
arr = np.arange(0, len(_df.dropna()), N_agents)
array_Price_values = _df.dropna().take(arr).Price.values

array_Step_values = _df.dropna().take(arr).Step.values

for el in it:
    
    _df = _[ _['iteration'] == el].drop(columns = ['iteration'])
    arr = np.arange(0, len(_df.dropna()), N_agents)
    array_Price_values_new = _df.dropna().take(arr).Price.values
    
    array_Price_values = np.vstack((array_Price_values, array_Price_values_new))

In [ ]:
mean_Price_values = np.mean(array_Price_values, axis = 0)
std_Price_values = np.std(array_Price_values, axis = 0)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure()

fig, ax = plt.subplots(figsize=(6, 6))

ax.plot(array_Step_values, mean_Price_values, alpha=.3, label= 'mean on 10 iterations', color='blue')
ax.set(title = 'Generated Prices, savers_sentiment: {}'.format(results_df['savers_sentiment'].unique()[2]),
            xlabel = 'Step', ylabel = 'Price')
ax.fill_between(
    array_Step_values,
    mean_Price_values - std_Price_values,
    mean_Price_values + std_Price_values,
    alpha=0.1,
    color="navy",
    lw=1,
)

plt.legend()
plt.show()